# 🧪 AI Neighborhood Curator 데이터 가이드 (v3.2)

이 노트북은 원본 데이터를 시스템의 **결정 레이어(Decision Layer)**와 **설명 레이어(Explanation Layer)**에서 즉시 사용할 수 있는 표준 CSV 포맷으로 변환하는 가이드를 제공합니다.

### 📜 데이터 헌법 준수 사항
1. **한글_언더바**: 모든 최종 컬럼명은 `행정동_명칭`, `보증금_만원` 등의 형식을 유지합니다.
2. **계층 분리**: 순위 결정용 데이터와 시그널 설명용 데이터를 철저히 분리하여 저장합니다.

In [ ]:
import pandas as pd
import numpy as np
import os

# 데이터 저장 경로 설정
RAW_DIR = '../data/raw'
PROCESSED_DIR = '../data/processed'
os.makedirs(PROCESSED_DIR, exist_ok=True)

## 1. 결정 레이어: 부동산 데이터 (Real Estate)
사용자의 예산에 맞는 매물 순위를 결정하는 핵심 데이터입니다.

In [ ]:
rent_file = os.path.join(RAW_DIR, 'seoul_real_estate_rent.csv')
df_rent = pd.read_csv(rent_file, encoding='utf-8-sig')

# 컬럼 표준화 (한글_언더바 규칙)
df_rent['행정동_명칭'] = df_rent['자치구명'] + " " + df_rent['법정동명']

mapping = {
    '건물명': '건물_명칭',
    '전월세구분': '전월세_구분',
    '보증금(만원)': '보증금_만원',
    '임대료(만원)': '임대료_만원'
}

df_rent_std = df_rent.rename(columns=mapping)

# 수치 보정 및 환산 월세 계산
df_rent_std['보증금_만원'] = pd.to_numeric(df_rent_std['보증금_만원'], errors='coerce').fillna(0)
df_rent_std['임대료_만원'] = pd.to_numeric(df_rent_std['임대료_만원'], errors='coerce').fillna(0)
df_rent_std['환산_월세'] = (df_rent_std['보증금_만원'] * 0.055 / 12) + df_rent_std['임대료_만원']

final_rent = df_rent_std[['행정동_명칭', '건물_명칭', '전월세_구분', '보증금_만원', '임대료_만원', '환산_월세']]
final_rent.to_csv(os.path.join(PROCESSED_DIR, 'rent_decision_layer.csv'), index=False, encoding='utf-8-sig')
print("✅ 부동산 결정 데이터 생성 완료!")

## 2. 결정 레이어: 통신 빅데이터 (Telecom)
동네의 활력, 안전, 출퇴근 시간을 결정하는 핵심 지표입니다.

In [ ]:
telecom_file = os.path.join(RAW_DIR, '2025.11월_29개 통신정보.xlsx')
df_telecom = pd.read_excel(telecom_file)

# 특수값 처리 및 행정동 통합
df_telecom = df_telecom.replace('*', 1.0)
df_telecom['행정동_명칭'] = df_telecom['자치구'] + " " + df_telecom['행정동']

# 주요 지표 피벗/집계
agg_rules = {
    '1인가구수': 'sum',
    '배달 서비스 사용일수': 'mean',
    '평균 출근 소요시간 평균': 'mean',
    '야간상주지 변경횟수 평균': 'mean'
}

df_telecom_agg = df_telecom.groupby('행정동_명칭').agg(agg_rules).reset_index()

# 점수 엔진용 컬럼명 표준화
df_telecom_agg = df_telecom_agg.rename(columns={
    '1인가구수': '일인가구_수',
    '배달 서비스 사용일수': '배달_사용지수',
    '평균 출근 소요시간 평균': '출퇴근_소요시간',
    '야간상주지 변경횟수 평균': '안전_변동지수'
})

df_telecom_agg.to_csv(os.path.join(PROCESSED_DIR, 'telecom_decision_layer.csv'), index=False, encoding='utf-8-sig')
print("✅ 통신 결정 데이터 생성 완료!")

## 3. 설명 레이어: 관심집단 데이터 (Interest Groups)
결정된 순위에 대해 '정성적인 해설'을 덧붙이기 위한 보조 데이터입니다.

In [ ]:
interest_file = os.path.join(RAW_DIR, '2025.11월_10개 관심집단수.xlsx')
df_interest = pd.read_excel(interest_file)

df_interest['행정동_명칭'] = df_interest['자치구'] + " " + df_interest['행정동']

# 설명에 사용할 핵심 관심 키워드 필터링 (예시)
df_interest.to_csv(os.path.join(PROCESSED_DIR, 'interest_explanation_layer.csv'), index=False, encoding='utf-8-sig')
print("✅ 설명 레이어 데이터 생성 완료!")